In [1]:
import torch
import torch.nn as nn # NN networks (CNN, RNN, losses)
import torch.optim as optim # Aptimizers (Adam, Adadelta, Adagrad)
import torch.nn.functional as F # Activarions func (ReLU, Sigmoid) also included in nn
from torch.utils.data import DataLoader # Dataset manager
import torchvision.datasets as datasets # Datasets
import torchvision.transforms as transforms # Transformation to datasets


In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparametrs
in_channels = 1
num_classes = 10
learning_rate = 0.001
num_epoches = 5
batch_size = 64


In [3]:
# Load data
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [4]:
from turtle import forward


class CNN(nn.Module):
    def __init__(self, in_channesls=1, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)
        

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        x = x.reshape(x.shape[0], -1)
        
        x = self.fc1(x)
        return x

In [5]:
# Initialize model
model = CNN().to(device=device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [6]:
# Train model
for epoch in range(num_epoches):
    for batch_ind, (data, targets) in enumerate(train_loader):
        # Data on cude
        data = data.to(device=device) # ([64, 1, 28, 28])
        targets = targets.to(device=device)
        
        # Forawrd
        scores = model(data) # Equal to model.forward(data)
        loss = criterion(scores, targets)

        # Backprop
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or adam step
        optimizer.step()


In [7]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    print(f"{float(num_correct) / float(num_samples)}")
    model.train()

In [8]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


0.98705
0.9858
